### Расчет MDE и размера выборки

In [1]:
#Нужно расчитать выборочное среднее и дисперсию
#Используем дельта-метод для расчета дисперсии, если у нас ratio-метрика (ниже)
def var_ratio(x,y): #x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]

    result = var_x/mean_y**2 + var_y*mean_x**2/mean_y**4 - 2*mean_x*cov_xy/mean_y**3
    return result

In [3]:
# минимальный наблюдаемый эффект
def get_MDE(mu, std, sample_size, n_groups=2, target_share=0.5, r=1, alpha=0.05, beta=0.2):
    """Возвращает MDE для обычной пользовательской метрики, при заданных параметрах теста.
                                    ТАРГЕТНЫЕ ГРУППЫ ДОЛЖНЫ БЫТЬ РАВНЫ
    mu: float, среднее выборки на исторических данных
    std: float, стан. отклонение выборки на исторических данных
    sample size: int, размер выборки для теста (включает в себя все группы)
    n_groups: int, количество групп в тесте с учетом всех контрольных и таргетных
    target_share: float, доля одной таргетной группы
    r: float, отношение самой маленькой группы к самой большой группе
    alpha: float, уровень ошибки I рода
    beta: float, уровень ошибки II рода

    return: MDE abs и MDE в %
    """

    t_alpha = stats.norm.ppf(1 - ((alpha / 2)), loc=0, scale=1)
    comparisons = n_groups - 1
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    sample_ratio_correction = r+2+1/r

    mde = np.sqrt(sample_ratio_correction)*(t_alpha + t_beta) * std / np.sqrt(
        sample_size*(1-target_share*(comparisons-1)))
    # mde = np.sqrt((r+1)/r)*(t_alpha + t_beta) * std / np.sqrt(sample_size/n_groups)

    return mde, mde*100/mu

In [4]:
# mde
get_MDE(mean_aa, np.sqrt(var_aa), sample_size_final)

NameError: name 'mean_aa' is not defined

In [5]:
# минимальный размер аудитории
def get_sample_size(mu, var, mde, r=1, alpha=0.05, beta=0.2):
    """Возвращает минимальный размер аудитории для обычной пользовательской метрики
        при заданных параметрах теста
    mu: float, среднее выборки на исторических данных
    var: float, дисперсия выборки на исторических данных
    mde: float, минимальный эффект
    r: float, отношение самой маленькой группы к самой большой группе
    alpha: float, уровень ошибки I рода
    beta: float, уровень ошибки II рода

    return: размер аудитории для одной группы
    """

    t_alpha = stats.norm.ppf(1 - (alpha / 2), loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)

    sample_size = (r+1) / r * var * (t_alpha + t_beta)**2 / (mde*mu)**2
    # sample_size = (r+1) / r * var * (t_alpha + t_beta)**2 / (mde)**2 # mde abs

    return sample_size